 # WWH BDT Training Notebook
 
 
 
 ## Importing and Variables

In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_curve
import json
import tqdm
import math
import os

from IPython.display import display

import bdt_drawing as bdraw
import bdt_training as btrain


In [ ]:
branches_tmp = ["Sample_name",
                "genWeight",
                "Xsec_genWeight",
                "EventType_SemiLeptonic",
                "EventType_Leptonic",
                "CandidateVBF_Lead_Jet_pt",
                "CandidateVBF_Trail_Jet_pt",
                "CandidateVBF_Jet_invMass",
                "CandidateVBF_Jet_etaSep",
                "CandidateHiggs_FatJet_pt",
                "CandidateHiggs_FatJet_eta",
                "CandidateHiggs_FatJet_msoftdrop",
                "CandidateHiggs_FatJet_mass",
                "CandidateHiggs_FatJet_particlenetScore",
                "CandidateW_SemiLeptonic_FatJet_pt",
                "CandidateW_SemiLeptonic_FatJet_eta",
                "CandidateW_SemiLeptonic_FatJet_msoftdrop",
                "CandidateW_SemiLeptonic_FatJet_mass",
                "CandidateW_SemiLeptonic_FatJet_particlenetScore",
                "CandidateW_SemiLeptonic_FatJet_flavor",
                "Candidate_SemiLeptonic_Lepton_pt",
                "Candidate_SemiLeptonic_Lepton_eta",
                "MET_pt",
                "Candidate_Leptonic_Lead_Lepton_pt",
                "Candidate_Leptonic_Trail_Lepton_pt",
                "Candidate_Leptonic_Lead_Lepton_eta",
                "Candidate_Leptonic_Trail_Lepton_eta",
                "Candidate_Leptonic_Lead_Lepton_phi",
                "Candidate_Leptonic_Trail_Lepton_phi",
                "Candidate_SemiLeptonic_ST",
                "Candidate_Leptonic_ST",
                "Candidate_Leptonic_LT",
                "Candidate_SemiLeptonic_LT",
                "Candidate_Leptonic_MT",
                "Candidate_SemiLeptonic_MT",
                "Candidate_SemiLeptonic_RpT",
                "Candidate_Leptonic_RpT",
                #"FatJet_particleNet_WvsQCD",
                #"CandidateW_SemiLeptonic_FatJet_idx"
                "Candidate_Leptonic_Lead_Lepton_type",
                "Candidate_Leptonic_Trail_Lepton_type",
                "Candidate_SemiLeptonic_Lepton_type",
                "Candidate_Leptonic_Lepton_InvMass"
               ]

eos_path = "/eos/cms/store/user/lzygala/HVV/Selection_TTrees_v2/AllYears/BDT_Settings_01232023/"

display_columns = ["name","year","xsec","nRawEvents","nScaledEvents","included"]
df_displayinfo_l = pd.DataFrame(columns = display_columns)
df_displayinfo_s = pd.DataFrame(columns = display_columns)

dfs_l = []
dfs_s = []

dfs_sig_l = {}
dfs_sig_s = {}

cut_l = "CandidateVBF_Jet_etaSep >= 4 and Candidate_Leptonic_ST > 1000"
cut_s = "CandidateVBF_Jet_etaSep >= 4 and Candidate_SemiLeptonic_ST > 1000"

years = ["16APV", "16", "17", "18"]
lumis = {
    "16APV" : 19.52, 
    "16" : 16.81, 
    "17" : 41.48, 
    "18" : 59.83
}

## Load data

All events are scaled by genWeight * xs * lumi / sum of genWeights
- Run 2 luminosity = 137.65

Single signal file is expected

### Load signal file:

In [ ]:
signal_types = ["C2V_0"
               ]
signal_xs = {"WWH_ucsd_C2V_Reweight" : 0.00243354,
             "WZH_ucsd_C2V_Reweight" : 0.00155167
            }

In [ ]:
sig_file_pre = "SIG_WWH_WZH_"
bkg_file_pre = "BKG_"

In [ ]:
for year in years:
    f_signal = eos_path + sig_file_pre + year + ".root"
    print(f_signal)
    
    f_open_signal_l = uproot.open(f_signal+":Events_leptonic")
    f_open_signal_s = uproot.open(f_signal+":Events_semileptonic")
    f_open_signal_cutflow = uproot.open(f_signal+":CutFlow")

    df_signal_s = f_open_signal_s.arrays(branches_tmp,library="pd")
    df_signal_l = f_open_signal_l.arrays(branches_tmp,library="pd")
    
    df_signal_s["sig_type"] = "C2V_Reweight"#signal_sample[9:]
    df_signal_s["abs_xsec_genWeight"] = np.abs(df_signal_s["Xsec_genWeight"])
    df_signal_s["sample_isSignal"] = True
    df_signal_s["year"] = year

    df_signal_l["sig_type"] = "C2V_Reweight"#signal_sample[9:]
    df_signal_l["abs_xsec_genWeight"] = np.abs(df_signal_l["Xsec_genWeight"])
    df_signal_l["sample_isSignal"] = True
    df_signal_l["year"] = year

    #df_displayinfo_l.loc[len(df_displayinfo_l.index)] = [signal_sample, year, signal_xs[signal_sample], len(df_signal_l), df_signal_l["abs_xsec_genWeight"].sum(),True]
    #df_displayinfo_s.loc[len(df_displayinfo_s.index)] = [signal_sample, year, signal_xs[signal_sample], len(df_signal_s), df_signal_s["abs_xsec_genWeight"].sum(),True]

    df_signal_s = df_signal_s[df_signal_s.eval(cut_s)].copy()
    df_signal_l = df_signal_l[df_signal_l.eval(cut_l)].copy()

    dfs_l.append(df_signal_l)
    dfs_s.append(df_signal_s)

    #dfs_sig_l[signal_sample].append(df_signal_l)
    #dfs_sig_s[signal_sample].append(df_signal_s)


In [ ]:
display(dfs_s)

### Load background files:

In [ ]:
info_backgrounds = json.load(open("bkg_info.json"))

dfs_l_background = []
dfs_s_background = []

for year in years:
    #print(sample["name"], year)
    filename = eos_path + bkg_file_pre + year +".root"
    print(filename)

    f_open_background_s = uproot.open(filename+":Events_semileptonic")
    f_open_background_l = uproot.open(filename+":Events_leptonic")
    f_open_background_cutflow = uproot.open(filename+":CutFlow")

    df_background_s = f_open_background_s.arrays(branches_tmp,library="pd")
    df_background_l = f_open_background_l.arrays(branches_tmp,library="pd")

    df_background_s = df_background_s[df_background_s.eval(cut_s)].copy()
    df_background_l = df_background_l[df_background_l.eval(cut_l)].copy()

    df_background_s["sig_type"] = "bkg"
    df_background_s["abs_xsec_genWeight"] = np.abs(df_background_s["Xsec_genWeight"])
    df_background_s["sample_isSignal"] = False
    df_background_s["year"] = year

    df_background_l["sig_type"] = "bkg"
    df_background_l["abs_xsec_genWeight"] = np.abs(df_background_l["Xsec_genWeight"])
    df_background_l["sample_isSignal"] = False
    df_background_l["year"] = year

    if df_background_l.empty == False:
        dfs_l.append(df_background_l)
        dfs_l_background.append(df_background_l)
        #df_displayinfo_l.loc[len(df_displayinfo_l.index)] = [sample["name"], year,  sample["xs"], len(df_background_l), df_background_l["abs_xsec_genWeight"].sum(),True]

    if df_background_s.empty == False:
        dfs_s.append(df_background_s)
        dfs_s_background.append(df_background_s)
        #df_displayinfo_s.loc[len(df_displayinfo_s.index)] = [sample["name"],year,  sample["xs"], len(df_background_s), df_background_s["abs_xsec_genWeight"].sum(),True]

display(df_displayinfo_l)
display(df_displayinfo_s)


## Split Samples for Training and Testing

Each sample is split:
- 75% of raw events for training
- 25% of raw events for testing

The training and testing sets then have their weights rescaled so each set has an integral equal to the total

In [ ]:
def split_samples(df, use_abs=True):
    split_columns = ["name","year","total_integral","test_integral","train_integral"]
    split_columns_info = ["name","year","total_integral","test_integral","train_integral"]
    df_split_info = pd.DataFrame(columns = split_columns)
    
    dfs = []
    
    for name in df.Sample_name.unique():
        for year in df.year.unique():
            df_tmp = df[(df.year==year) & (df.Sample_name == name)].copy()
            
            if df_tmp.empty == True:
                continue
            df_tmp["split_train"] = (np.random.rand(len(df_tmp)) < 0.75)
            split_train = df_tmp["split_train"]
            split_test = ~df_tmp["split_train"]

            total, split_test_integral, split_train_integral = 0, 0, 0

            if use_abs:
                df_tmp["split_weight"] = df_tmp.abs_xsec_genWeight
                total = df_tmp.abs_xsec_genWeight.sum()
                split_test_integral = df_tmp[split_test].abs_xsec_genWeight.sum()
                split_train_integral = df_tmp[split_train].abs_xsec_genWeight.sum()
            else:
                df_tmp["split_weight"] = df_tmp.xsec_genWeight
                total = df_tmp.Xsec_genWeight.sum()
                split_test_integral = df_tmp[split_test].Xsec_genWeight.sum()
                split_train_integral = df_tmp[split_train].Xsec_genWeight.sum()

            df_split_info.loc[len(df_split_info.index)] = [name, year, total, split_test_integral, split_train_integral]


            if split_train_integral != 0 and split_test_integral != 0:
                df_tmp.loc[split_train, "split_weight"] *= total/split_train_integral
                df_tmp.loc[split_test, "split_weight"] *= total/split_test_integral
            else:
                #print(df.iloc[-1].at["sample_name"]+" 0 occured!")
                #display(df[["split_train"]])
                df_tmp.loc[split_train, "split_weight"] *= 0
                df_tmp.loc[split_test, "split_weight"] *= 0
            
            dfs.append(df_tmp)

    df_combined = pd.concat(dfs)
    display(df_split_info)
    return df_combined

In [ ]:
df_s_tmp = pd.concat(dfs_s)
df_l_tmp = pd.concat(dfs_l)

df_combined_s_LepType = split_samples(df_s_tmp)
df_combined_l_LepType = split_samples(df_l_tmp)

## Set Up Model and Train

### Variables to be trained on:

In [ ]:
df_combined_l_LepType["Lep_dPhi"] = np.abs(df_combined_l_LepType["Candidate_Leptonic_Lead_Lepton_phi"] - df_combined_l_LepType["Candidate_Leptonic_Trail_Lepton_phi"])
df_combined_l_LepType.loc[df_combined_l_LepType.Lep_dPhi > 3.14159, "Lep_dPhi"] = df_combined_l_LepType.loc[df_combined_l_LepType.Lep_dPhi > 3.14159, "Lep_dPhi"] - (2*3.14159)

df_combined_l_LepType["Lep_dEta"] = np.abs(df_combined_l_LepType["Candidate_Leptonic_Lead_Lepton_eta"] - df_combined_l_LepType["Candidate_Leptonic_Trail_Lepton_eta"])
df_combined_l_LepType["Lep_dR"] = np.sqrt(np.power(df_combined_l_LepType["Lep_dPhi"], 2) + np.power(df_combined_l_LepType["Lep_dEta"],2))

features_s_LepType = {
    "CandidateVBF_Jet_invMass" : "f0",
    "CandidateVBF_Jet_etaSep" : "f1",
    "CandidateVBF_Lead_Jet_pt" : "f2",
    "CandidateVBF_Trail_Jet_pt" : "f3",
    "CandidateHiggs_FatJet_pt" : "f4",
    "CandidateHiggs_FatJet_eta" : "f5",
    "CandidateHiggs_FatJet_msoftdrop" : "f6",
    #"CandidateHiggs_FatJet_particlenetScore",
    "CandidateW_SemiLeptonic_FatJet_pt" : "f7",
    "CandidateW_SemiLeptonic_FatJet_eta" : "f8",
    "CandidateW_SemiLeptonic_FatJet_msoftdrop" : "f9",
    "CandidateW_SemiLeptonic_FatJet_flavor" : "f10",
    "Candidate_SemiLeptonic_Lepton_pt" : "f11",
    "Candidate_SemiLeptonic_Lepton_eta" : "f12",
    "MET_pt" : "f13",
    "Candidate_SemiLeptonic_ST" : "f14",
    #"Candidate_SemiLeptonic_RpT"
    "EventType_SemiLeptonic" : "f15"
}

features_l_LepType = {
    "CandidateVBF_Jet_invMass" : "f0",
    "CandidateVBF_Jet_etaSep" : "f1",
    "CandidateVBF_Lead_Jet_pt" : "f2",
    "CandidateVBF_Trail_Jet_pt" : "f3",
    "CandidateHiggs_FatJet_pt" : "f4",
    "CandidateHiggs_FatJet_eta" : "f5",
    "CandidateHiggs_FatJet_msoftdrop" : "f6",
    #"CandidateHiggs_FatJet_particlenetScore",
    "Candidate_Leptonic_Lead_Lepton_pt" : "f7",
    "Candidate_Leptonic_Trail_Lepton_pt" : "f8",
    "Candidate_Leptonic_Lead_Lepton_eta" : "f9",
    "Candidate_Leptonic_Trail_Lepton_eta" : "f10",
    "Candidate_Leptonic_Lepton_InvMass" : "f11",
    "MET_pt" : "f12",
    "Candidate_Leptonic_ST" : "f13",
    "Lep_dR" : "f14",
    "Lep_dEta" : "f15",
    "Candidate_Leptonic_MT" : "f16",
    "EventType_Leptonic" : "f17"
    #"Candidate_Leptonic_RpT"
}

In [ ]:
bdts_s_LepType = {}
bdts_l_LepType = {}
eta_s = {
        "16APV" : 0.464, 
        "16" : 0.543, 
        "17" : 0.478, 
        "18" : 0.732
        }

n_s = {
        "16APV" : 1, 
        "16" : 1, 
        "17" : 10, 
        "18" : 6
        }

mcw_s = {
        "16APV" : 1, 
        "16" : 1, 
        "17" : 7, 
        "18" : 4
        }

eta_l = {
        "16APV" : 0.98, 
        "16" : 0.92, 
        "17" : 0.618, 
        "18" : 0.506
        }

n_l = {
        "16APV" : 10, 
        "16" : 6, 
        "17" : 10, 
        "18" : 2
        }

mcw_l = {
        "16APV" : 0, 
        "16" : 0, 
        "17" : 0, 
        "18" : 0
        }

#for sig_name in tqdm.tqdm(signal_xs):
#for sig_name in tqdm.tqdm(["WWH_ucsd_C2V_Reweight"]):
for sig_name in ["WWH_ucsd_C2V_Reweight"]:
    print("TRAINING: ", sig_name)
    bdts_s_LepType[sig_name] = {}
    bdts_l_LepType[sig_name] = {}

    for year in years:
        
        print(year, " 1L: ", end=" ")
        bdts_s_LepType[sig_name][year] = btrain.train_bdt(df_combined_s_LepType,features_s_LepType,year,sig_name,n_s[year],eta_s[year],mcw=mcw_s[year],verb=False)
        print(year, " 2L: ", end=" ")
        bdts_l_LepType[sig_name][year] = btrain.train_bdt(df_combined_l_LepType,features_l_LepType,year,sig_name,n_l[year],eta_l[year],mcw=mcw_l[year],verb=False)
print()
signal_types = ["C2V_Reweight"]

#for sig_name in tqdm.tqdm(signal_types):
for sig_name in signal_types:
    print("TRAINING: ", signal_types)
    bdts_s_LepType[sig_name] = {}
    bdts_l_LepType[sig_name] = {}
    
    for year in years:
        
        print(year, " 1L: ", end=" ")
        bdts_s_LepType[sig_name][year] = btrain.train_bdt_mixed(df_combined_s_LepType,features_s_LepType,year,sig_name,n_s[year],eta_s[year],mcw=mcw_s[year],verb=False)
        
        print(year, " 2L: ", end=" ")
        bdts_l_LepType[sig_name][year] = btrain.train_bdt_mixed(df_combined_l_LepType,features_l_LepType,year,sig_name,n_l[year],eta_l[year],mcw=mcw_l[year],verb=False)
    

Save BDTs to files:

In [ ]:
import ROOT
for year in years:
    ROOT.TMVA.Experimental.SaveXGBoost(bdts_s_LepType["C2V_Reweight"][year], "bdt_1L_"+year, "bdt_1L_"+year+".root")
    ROOT.TMVA.Experimental.SaveXGBoost(bdts_l_LepType["WWH_ucsd_C2V_Reweight"][year], "bdt_2L_"+year, "bdt_2L_"+year+".root")

## Check Training Results

### Show ranking of variables:

In [ ]:
full_bdt_name_list = ["C2V_Reweight",
                      "WWH_ucsd_C2V_Reweight"
                     ]

In [ ]:
for sig_name in full_bdt_name_list:
    print(sig_name)
    for year in years:
        print(year)
        xgb.plot_importance(bdts_l_LepType[sig_name][year], importance_type="gain")
        plt.show()

In [ ]:
for sig_name in full_bdt_name_list:
    print(sig_name)
    for year in years:
        print(year)
        xgb.plot_importance(bdts_s_LepType[sig_name][year], importance_type="gain")
        plt.show()

## Apply Training Results to Samples and Plot

### Apply Training Predictions:

In [ ]:
#Apply predictions to WZH and WWH trained together

df_SigSampsMixed_l_LepType_dict = {}
df_SigSampsMixed_s_LepType_dict = {}

features_s_LepType_inv = {v: k for k, v in features_s_LepType.items()}
features_l_LepType_inv = {v: k for k, v in features_l_LepType.items()}

for sig_name in full_bdt_name_list:
    sig_type = sig_name
    if "ucsd" in sig_type:
        sig_type = sig_type[9:]
        
    
    df_SigSampsMixed_l_LepType_dict[sig_name] = {}
    df_SigSampsMixed_s_LepType_dict[sig_name] = {}

    for year in years:
        
        df_SigSampsMixed_l_LepType_dict[sig_name][year] = df_combined_l_LepType[(df_combined_l_LepType.year==year) & ((df_combined_l_LepType.sample_isSignal == 0) | (df_combined_l_LepType.sig_type == sig_type))].copy()
        df_SigSampsMixed_s_LepType_dict[sig_name][year] = df_combined_s_LepType[(df_combined_s_LepType.year==year) & ((df_combined_s_LepType.sample_isSignal == 0) | (df_combined_s_LepType.sig_type == sig_type))].copy()

        df_SigSampsMixed_s_LepType_dict[sig_name][year].rename(columns=features_s_LepType, inplace=True)
        df_SigSampsMixed_l_LepType_dict[sig_name][year].rename(columns=features_l_LepType, inplace=True)

        
        df_SigSampsMixed_l_LepType_dict[sig_name][year]["bdt"] = (bdts_l_LepType[sig_name][year].predict_proba(df_SigSampsMixed_l_LepType_dict[sig_name][year][list(features_l_LepType.values())]))[:,1]
        df_SigSampsMixed_s_LepType_dict[sig_name][year]["bdt"] = (bdts_s_LepType[sig_name][year].predict_proba(df_SigSampsMixed_s_LepType_dict[sig_name][year][list(features_s_LepType.values())]))[:,1]
        #display(df_SigSampsMixed_s_LepType_dict[sig_name][year])
        
        df_SigSampsMixed_s_LepType_dict[sig_name][year].rename(columns=features_s_LepType_inv, inplace=True)
        df_SigSampsMixed_l_LepType_dict[sig_name][year].rename(columns=features_l_LepType_inv, inplace=True)
    

In [ ]:
test_vals = ["C2V_Reweight"
            ]

df_SigSampTrainingCompare_l_LepType_dict = {}
df_SigSampTrainingCompare_s_LepType_dict = {}

for val in test_vals:
    df_SigSampTrainingCompare_l_LepType_dict[val] = {"WWH" : pd.concat([df_SigSampsMixed_l_LepType_dict["WWH_ucsd_" + val]["16"],
                                                               df_SigSampsMixed_l_LepType_dict["WWH_ucsd_" + val]["17"],
                                                               df_SigSampsMixed_l_LepType_dict["WWH_ucsd_" + val]["18"],
                                                               df_SigSampsMixed_l_LepType_dict["WWH_ucsd_" + val]["16APV"]]),
                                                     "BOTH" : pd.concat([df_SigSampsMixed_l_LepType_dict[val]["16"],
                                                               df_SigSampsMixed_l_LepType_dict[val]["17"],
                                                               df_SigSampsMixed_l_LepType_dict[val]["18"],
                                                               df_SigSampsMixed_l_LepType_dict[val]["16APV"]])}
    
    
    
    df_SigSampTrainingCompare_s_LepType_dict[val] = {"WWH" : pd.concat([df_SigSampsMixed_s_LepType_dict["WWH_ucsd_" + val]["16"],
                                                               df_SigSampsMixed_s_LepType_dict["WWH_ucsd_" + val]["17"],
                                                               df_SigSampsMixed_s_LepType_dict["WWH_ucsd_" + val]["18"],
                                                               df_SigSampsMixed_s_LepType_dict["WWH_ucsd_" + val]["16APV"]]),
                                                     "BOTH" : pd.concat([df_SigSampsMixed_s_LepType_dict[val]["16"],
                                                               df_SigSampsMixed_s_LepType_dict[val]["17"],
                                                               df_SigSampsMixed_s_LepType_dict[val]["18"],
                                                               df_SigSampsMixed_s_LepType_dict[val]["16APV"]])}
    
    
    
    

### Show ROC:

In [ ]:
for sig_name in full_bdt_name_list:
    print(sig_name)
    for year in years:
        print(year)
        df_tmp = df_SigSampsMixed_s_LepType_dict[sig_name][year].copy()
        df_train = df_tmp[(df_tmp.split_train) & (df_tmp.year==year)].copy()
        df_test = df_tmp[(~df_tmp.split_train) & (df_tmp.year==year)].copy()
        
        fpr, tpr, thresh = roc_curve(df_train.sample_isSignal, df_train["bdt"])
        plt.plot(fpr, tpr, label="Training : AUC = {:.2f}".format(np.trapz(tpr,fpr)));
        
        fpr, tpr, thresh = roc_curve(df_test.sample_isSignal, df_test["bdt"])
        plt.plot(fpr, tpr, label="Testing : AUC = {:.2f}".format(np.trapz(tpr,fpr)));
        
        plt.xlabel("background efficiency", size=18);
        plt.ylabel("signal efficiency", size=18);
        plt.legend(fontsize=16);

        plt.show()

In [ ]:
for sig_name in full_bdt_name_list:
    print(sig_name)
    for year in years:
        print(year)
        df_tmp = df_SigSampsMixed_l_LepType_dict[sig_name][year].copy()
        df_train = df_tmp[(df_tmp.split_train) & (df_tmp.year==year)].copy()
        df_test = df_tmp[(~df_tmp.split_train) & (df_tmp.year==year)].copy()
        
        fpr, tpr, thresh = roc_curve(df_train.sample_isSignal, df_train["bdt"])
        plt.plot(fpr, tpr, label="Training : AUC = {:.2f}".format(np.trapz(tpr,fpr)));
        
        fpr, tpr, thresh = roc_curve(df_test.sample_isSignal, df_test["bdt"])
        plt.plot(fpr, tpr, label="Testing : AUC = {:.2f}".format(np.trapz(tpr,fpr)));
        
        plt.xlabel("background efficiency", size=18);
        plt.ylabel("signal efficiency", size=18);
        plt.legend(fontsize=16);

        plt.show()

### Draw BDT Scores:

In [ ]:
bdraw.draw_bdt_score_quant(df_SigSampTrainingCompare_s_LepType_dict["C2V_Reweight"]["BOTH"])

In [ ]:
bdraw.draw_bdt_score_quant(df_SigSampTrainingCompare_l_LepType_dict["C2V_Reweight"]["WWH"])

In [ ]:
df_test_s = df_SigSampTrainingCompare_s_LepType_dict["C2V_Reweight"]["BOTH"].copy()

df_cut = df_test_s[(df_test_s.CandidateHiggs_FatJet_particlenetScore > 0.9) 
                                   & (df_test_s.CandidateW_SemiLeptonic_FatJet_particlenetScore > 0.9)
                                   & (df_test_s.Candidate_SemiLeptonic_ST > 1000)
                                  ].copy()

bdraw.draw_bdt_score_quant(df_cut)

In [ ]:
df_test_l = (df_SigSampTrainingCompare_l_LepType_dict["C2V_Reweight"]["WWH"]).copy()


df_cut = df_test_l[(df_test_l.CandidateHiggs_FatJet_particlenetScore > 0.9) 
                                   & (df_test_l.Candidate_Leptonic_ST > 1000)
                                  ].copy()

bdraw.draw_bdt_score_quant(df_cut)

Send values for c2v to be drawn

In [ ]:
bdraw.draw_bdt_score_overlay(df_SigSampTrainingCompare_l_LepType_dict["C2V_Reweight"])

In [ ]:
bdraw.draw_bdt_score_overlay(df_SigSampTrainingCompare_s_LepType_dict["C2V_Reweight"])

### Draw Variables with BDT Score:

In [ ]:
bdt_cut_s = 0.95
fontsize=8
other_cut = ""
df_test = df_SigSampTrainingCompare_s_LepType_dict["C2V_Reweight"]["BOTH"]

df_cut = df_test[(df_test.CandidateHiggs_FatJet_particlenetScore > 0.95) 
                                   & (df_test.CandidateW_SemiLeptonic_FatJet_particlenetScore > 0.95) 
                                   & (df_test.CandidateVBF_Jet_etaSep > 5.0)
                                  ].copy()
#df_cut = df_combined_trained_s_lep[(df_combined_trained_s_lep.CandidateHiggs_FatJet_particlenetScore > 0.9)].copy()

bdraw.draw_plots_tmp_2(df_cut,"CandidateVBF_Jet_invMass",bdt_cut_s,"VBF Inv Mass",np.linspace(0, 5000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateVBF_Jet_etaSep",bdt_cut_s,"VBF |dEta|",np.linspace(0, 10, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_pt",bdt_cut_s,"Higgs PT",np.linspace(0, 3000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_particlenetScore",bdt_cut_s,"Higgs PN Score",np.linspace(0, 1.0, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_mass",bdt_cut_s,"Higgs Mass",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_msoftdrop",bdt_cut_s,"Higgs msoftdrop",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateW_SemiLeptonic_FatJet_particlenetScore",bdt_cut_s,"W PN Score",np.linspace(0, 1.0, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateW_SemiLeptonic_FatJet_pt",bdt_cut_s,"W PT",np.linspace(0, 3000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateW_SemiLeptonic_FatJet_mass",bdt_cut_s,"W Mass",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateW_SemiLeptonic_FatJet_msoftdrop",bdt_cut_s,"W msoftdrop",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_SemiLeptonic_Lepton_pt",bdt_cut_s,"Lepton PT",np.linspace(0, 3000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"MET_pt",bdt_cut_s,"MET",np.linspace(0, 2500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_SemiLeptonic_ST",bdt_cut_s,"ST",np.linspace(0, 3500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_SemiLeptonic_LT",bdt_cut_s,"LT",np.linspace(0, 3500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_SemiLeptonic_RpT",bdt_cut_s,"RpT",np.linspace(0, 5, 101),fontsize)

In [ ]:
bdt_cut_l = 0.95
fontsize=8
other_cut = ""
df_test = df_SigSampTrainingCompare_l_LepType_dict["C2V_Reweight"]["WWH"]

df_cut = df_test[(df_test.CandidateHiggs_FatJet_particlenetScore > 0.95) 
                    & (df_test.CandidateVBF_Jet_etaSep > 5.0)
                                  ].copy()
#df_cut = df_combined_trained_s_lep[(df_combined_trained_s_lep.CandidateHiggs_FatJet_particlenetScore > 0.9)].copy()

bdraw.draw_plots_tmp_2(df_cut,"CandidateVBF_Jet_invMass",bdt_cut_s,"VBF Inv Mass",np.linspace(0, 5000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateVBF_Jet_etaSep",bdt_cut_s,"VBF |dEta|",np.linspace(0, 10, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_pt",bdt_cut_s,"Higgs PT",np.linspace(0, 3000, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_particlenetScore",bdt_cut_s,"Higgs PN Score",np.linspace(0, 1.0, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_mass",bdt_cut_s,"Higgs Mass",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"CandidateHiggs_FatJet_msoftdrop",bdt_cut_s,"Higgs msoftdrop",np.linspace(0, 500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"MET_pt",bdt_cut_s,"MET",np.linspace(0, 2500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_Leptonic_ST",bdt_cut_s,"ST",np.linspace(0, 3500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_Leptonic_LT",bdt_cut_s,"LT",np.linspace(0, 3500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_Leptonic_Lead_Lepton_pt",bdt_cut_s,"Lead Lepton PT",np.linspace(0, 3500, 101),fontsize)
bdraw.draw_plots_tmp_2(df_cut,"Candidate_Leptonic_Trail_Lepton_pt",bdt_cut_s,"Trail Lepton PT",np.linspace(0, 3500, 101),fontsize)



# HYPEROPT SECTION
Only needs to be performed once to determine hyperparameters

In [ ]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score
from functools import partial


In [ ]:
def train_bdt_mixed_ho(space, df, features, year, signal_type):
    if year != "":
        df_training = df[(df.split_train) & (df.year==year) & ((df.sample_isSignal == 0) | (df.sig_type == signal_type))].sample(frac=1.)
        df_testing = df[(~df.split_train) & (df.year==year) & ((df.sample_isSignal == 0) | (df.sig_type == signal_type))].sample(frac=1.)
    else:
        df_training = df[(df.split_train) & ((df.sample_isSignal == 0) | (df.sig_type == signal_type))].sample(frac=1.)
        df_testing = df[(~df.split_train) & ((df.sample_isSignal == 0) | (df.sig_type == signal_type))].sample(frac=1.)

    if(len(df_training.index) == 0 or len(df_testing.index) == 0):
        print("0 occurred: ",year)
        return
    
    X_train, y_train = df_training[features].copy(), df_training.sample_isSignal.values
    w_train = (np.abs(df_training.split_weight)).values
    X_test, y_test = df_testing[features].copy(), df_testing.sample_isSignal.values
    w_test = (np.abs(df_testing.split_weight)).values
    
    X_train.rename(columns=features, inplace=True)
    X_test.rename(columns=features, inplace=True)

    xgbd_testing = xgb.DMatrix(
        df_testing[features.keys()], 
        label=df_testing.sample_isSignal,
        weight=np.abs(df_testing.split_weight),
        nthread=-1
    )
    xgbd_training = xgb.DMatrix(
        df_training[features.keys()], 
        label=df_training.sample_isSignal, 
        weight=np.abs(df_training.split_weight),
        nthread=-1
    )
    #evallist = [(xgbd_training, "train"), (xgbd_testing, "eval")]

    nRounds = 500
    params = {}
    params["early_stopping_rounds"] =  10
    params["n_estimators"] =  500
    params["objective"] = "binary:logistic"
    params["eval_metric"] = "auc"
    params["verbosity"] = 1
    params["nthread"] = -1
    
    params["max_depth"] = int(space['max_depth'])
    params["learning_rate"] = space['learning_rate']
    params["min_child_weight"] = space['min_child_weight']
    #params["max_leaves"] = int(space['max_leaves'])
    #params["reg_alpha"] = space['reg_alpha']
    #params["reg_lambda"] = space['reg_lambda']

    sumWeights_training_signal = np.abs(xgbd_training.get_weight()[xgbd_training.get_label() == 1]).sum()
    sumWeights_training_background = np.abs(xgbd_training.get_weight()[xgbd_training.get_label() == 0]).sum()
    params["scale_pos_weight"] = sumWeights_training_background/sumWeights_training_signal

    xgb_reg = xgb.XGBClassifier(**params)
    xgb_reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], sample_weight = w_train, sample_weight_eval_set = [w_test], verbose=False, early_stopping_rounds=10)
    #xgb_reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], sample_weight = w_train,  verbose=True)
    
    pred = (xgb_reg.predict_proba(X_test))[:,1]
    accuracy = xgb_reg.best_score#accuracy_score(y_test, pred>0.7)#, sample_weight=w_test)
    #print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': xgb_reg}

def train_bdt_ho(space, df, features, year, sig_name):
            
    if year != "":
        df_training = df[(df.split_train) & (df.year==year) & ((df.sample_isSignal == 0) | (df.sample_name == sig_name))].sample(frac=1.)
        df_testing = df[(~df.split_train) & (df.year==year) & ((df.sample_isSignal == 0) | (df.sample_name == sig_name))].sample(frac=1.)
    else:
        df_training = df[(df.split_train) & ((df.sample_isSignal == 0) | (df.sample_name == sig_name))].sample(frac=1.)
        df_testing = df[(~df.split_train) & ((df.sample_isSignal == 0) | (df.sample_name == sig_name))].sample(frac=1.)

    split_test_integral = df_testing.xsec_genWeight.sum()
    split_train_integral = df_training.xsec_genWeight.sum()
    
    xgbd_testing = xgb.DMatrix(
        df_testing[features.keys()], 
        label=df_testing.sample_isSignal,
        weight=np.abs(df_testing.split_weight),
        nthread=-1
    )
    xgbd_training = xgb.DMatrix(
        df_training[features.keys()], 
        label=df_training.sample_isSignal, 
        weight=np.abs(df_training.split_weight),
        nthread=-1
    )
    
    X_train, y_train = df_training[features].copy(), df_training.sample_isSignal.values
    w_train = (np.abs(df_training.split_weight)).values
    X_test, y_test = df_testing[features].copy(), df_testing.sample_isSignal.values
    w_test = (np.abs(df_testing.split_weight)).values
    
    X_train.rename(columns=features, inplace=True)
    X_test.rename(columns=features, inplace=True)

    #display(df_training_l)

    #evallist = [(xgbd_training, "train"), (xgbd_testing, "eval")]

    nRounds = 500
    params = {}
    params["objective"] = "binary:logistic"
    params["eval_metric"] = "auc"
    params["verbosity"] = 1
    params["nthread"] = -1
    
    params["max_depth"] = int(space['max_depth'])
    params["learning_rate"] = space['learning_rate']
    #params["gamma"] = space['gamma']
    #params["max_leaves"] = int(space['max_leaves'])
    #params["reg_alpha"] = space['reg_alpha']
    #params["reg_lambda"] = space['reg_lambda']
    params["min_child_weight"] = space['min_child_weight']


    sumWeights_training_signal = np.abs(xgbd_training.get_weight()[xgbd_training.get_label() == 1]).sum()
    sumWeights_training_background = np.abs(xgbd_training.get_weight()[xgbd_training.get_label() == 0]).sum()
    params["scale_pos_weight"] = sumWeights_training_background/sumWeights_training_signal

    xgb_reg = xgb.XGBClassifier(**params)
    xgb_reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], sample_weight = w_train, sample_weight_eval_set = [w_test], verbose=False, early_stopping_rounds=10)
    
    pred = (xgb_reg.predict_proba(X_test))[:,1]
    accuracy = xgb_reg.best_score#accuracy_score(y_test, pred>0.7)#, sample_weight=w_test)
    #print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': xgb_reg}


In [ ]:

space={'max_depth': hp.quniform("max_depth", 0, 10, 1),
        'learning_rate': hp.uniform ('learning_rate', 0,1),
        #'gamma': hp.uniform ('gamma', 1,9),
        #'max_leaves' : hp.quniform('max_leaves', 0, 10, 1),
        #'reg_alpha' : hp.quniform('reg_alpha', 0,180,1),
        #'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1)
    }


In [ ]:
for year in years:
    print(year, ": ")
    fmin_objective_1L = partial(train_bdt_mixed_ho, df=df_combined_s_LepType, features=features_s_LepType, year=year, signal_type="C2V_Reweight")
    fmin_objective_2L = partial(train_bdt_ho, df=df_combined_l_LepType, features=features_l_LepType, year=year, sig_name="WWH_ucsd_C2V_Reweight")

    trials = Trials()
    best = fmin(fn=fmin_objective_1L,
                space=space,
                algo=tpe.suggest,
                max_evals=100,
                trials=trials)

    print (best)

  8%|▊         | 8/100 [00:46<07:26,  4.86s/trial, best loss: -0.962081]